### 문제 정의
서울 지역(구)의 경도, 위도 정보를 이용하여 임의로 입력된 지역(동)을
강동, 강서, 강남, 강북으로 분류

### 데이터 수집
서울의 대표적인 구(district)위치 데이터

컬럼 주석  
- district : 행정구역(서초구, 송파구....)  
- dong : 구(district)보다 작은 행정구역 (대치동, 도곡동....)  
- longitude : 경도  
- latitude : 위도  
- label : 한강 기준으로 동, 서, 남, 북으로 구분한 지역  명칭  

In [1]:
import pickle

In [22]:
with open('./Data/seoulmap/district_dict_list.pickle', 'rb') as f:
    train_df = pickle.load(f)
train_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './Data/seoulmap/district_dict_list.pickle'

In [3]:
with open('./Data/seoulmap/dong_dict_list.pickle', 'rb') as f:
    test_df = pickle.load(f)
test_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './Data/dong_dict_list.pickle'

In [4]:
import pandas as pd

In [5]:
# 현재 가지고 있는 데이터 레이블 갯수 확인
train_df.label.value_counts()

NameError: name 'train_df' is not defined

In [6]:
test_df.label.value_counts()

NameError: name 'test_df' is not defined

현재 데이터로 강북, 강서, 강동, 강남이라는 4가지 분류값을 가지고  
위도, 경도 정보만으로 지역의 레이블을 예측 해볼수 있도록 전처리.

### 데이터 전처리

In [7]:
# 경도, 위도 평균과, 편차
train_df.describe()

NameError: name 'train_df' is not defined

### 데이터 시각화

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
# 경도, 위도에 따른 데이터 시각화
sns.lmplot('longitude', 'latitude', data=train_df, fit_reg=False, 
          scatter_kws={"s": 150},
          markers=["o", 'x', "+", "*"],
          hue='label')
plt.title('district visualization in 2d plane')

NameError: name 'train_df' is not defined

강서는 동그라미, 강북은 +, 강남은 x, 강동은 별표  
강서는 중심으로부터 서쪽에, 강동은 중심으로부터 동쪽에  
강북은 중심으로부터 북쪽에, 강남은 중심으로부터 남쪽에 위치한 패턴

### 데이터 다듬기
학습에 필요없는 특징 데이터 제거  
학습 데이터에서는 구 이름 컬럼 제거  
테스트 데이터에서는 동 이름 컬럼 제거  

In [10]:
train_df.drop(['district'], axis=1, inplace=True)
test_df.drop(['dong'], axis=1, inplace=True)

NameError: name 'train_df' is not defined

In [11]:
X_train = train_df[['longitude', 'latitude']]
y_train = train_df[['label']]

NameError: name 'train_df' is not defined

In [12]:
X_test = test_df[['longitude', 'latitude']]
y_test = test_df[['label']]

NameError: name 'test_df' is not defined

### 결정 표면 시각화 함수

In [13]:
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [14]:
def display_desision_surface(clf, X, y):
    # 차트의 범위와 모든 학습 데이터를 포함하도록 설정
    x_min = X.longitude.min() - 0.01
    x_max = X.longitude.max() + 0.01
    y_min = X.latitude.min() - 0.01
    y_max = X.latitude.max() + 0.01
    
    #파라미터 설정
    n_classes = len(le.classes_)
    plot_colors = 'rywb'
    plot_step = 0.001
    
    # plt.contourf 차트를 그리기 위해 X, Y 이차원 영역 설정
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)
    
    # 학습 데이터를 차트에 표시
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X.loc[idx].longitude,
                   X.loc[idx].latitude,
                   c=color,
                   label=le.classes_[i],
                   cmap=plt.cm.RdYlBu, edgecolor='black', s=200)
    #차트 제목
    plt.title("Decision surface of a decision tree", fontsize=16)
    #차트 기호 설명
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=14)
    # x축의 이름과 폰트 크기 설정
    plt.xlabel('longitude', fontsize=16)
    # y축의 이름과 폰트 크기 설정
    plt.ylabel('latitude', fontsize=16)
    #차트 크기 설정
    plt.rcParams['figure.figsize'] = [7, 5]
    #차트 폰트 크기 설정
    plt.rcParams['font.size'] = 14
    #x축 좌표상의 폰트 크기 설정
    plt.rcParams['xtick.labelsize'] = 14
    #y축 좌표상의 폰트 크기 설정
    plt.rcParams['ytick.labelsize'] = 14
    #차트 그리기
    plt.show()

### 파라미터 없이 학습한 모델의 결정 표면 시각화
의사결정트리는 오버피팅되기 상당히 쉬운 모델.  
별도의 파라미터를 설정 하지 않을 경우 최대한 학습데이터에만 잘 맞게 모델이 형성(오버피팅)  
시각화 결과를 모면 강남, 강북, 강동, 강서 지역 부분이라기 보다는  
단순한 학습 데이터 구분에만 집중된 모델임을 볼 수 있다.

In [15]:
le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y_train)

clf = tree.DecisionTreeClassifier(random_state=35).fit(X_train, y_encoded)

NameError: name 'y_train' is not defined

In [16]:
display_desision_surface(clf, X_train, y_encoded)

NameError: name 'clf' is not defined

### 파라미터 설정한 모델의 결정 표면 시각화
- max_depth : 트리의 최대 한도 깊이  
- min_samples_split : 자식 노드를 갖기 위한 최소한의 데이터 갯수  
- mins_samples_leaf : 맨 마직막 끝 노드의 최소 데이터 갯수  
- random_state : 여러번 실행해도, 파라미터가 같을 경우 결과가 항상 같게 유지  
오버피팅을 피하기 위해 파라미터를 설정한 의사결정트리의 의사 결정 표면 차트
오버피팅된 이전 모델보다 강동, 강서, 강남 강북을 더적절하게 분류 한 것을 확인 할수있다.

In [17]:
clf = tree.DecisionTreeClassifier(max_depth=4,
                                 min_samples_split=2,
                                 min_samples_leaf=2,
                                 random_state=70).fit(X_train, y_encoded.ravel())

NameError: name 'X_train' is not defined

In [18]:
display_desision_surface(clf, X_train, y_encoded)

NameError: name 'clf' is not defined

### 의사결정트리 시각화

In [19]:
# !conda install -c anaconda graphviz
# !pip install graphviz
# https://graphviz.gitlab.io/_pages/Download/Download_windows.html

In [20]:
import graphviz

In [21]:
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("seoul")

dot_data = tree.export_graphviz(clf, out_file=None,
                               feature_names=['longitude', 'latitude'],
                               class_names=['Gangbuk', 'Gangdong', 'Gangnam', 'Gangseo'],
                               filled=True, rounded=True,
                               special_characters=True)
graph = graphviz.Source(dot_data)
graph

NameError: name 'clf' is not defined